# Transport Score Climbing
This tutorial demonstrates a recently developed method for Bayesian inference called Transport Score Climbing ([Zhang et al., 2022](https://arxiv.org/abs/2202.01841)). We explain how the method works, how to implement it using FlowTorch, and demonstrate a simple toy example, replicating Figure 3 from [the paper](https://arxiv.org/pdf/2202.01841.pdf), and comparing to standard variational inference.